# Query rewriting


We have focused in better chunking, better retrieval and better RAG techonology, but some times, queries are not very good

![bad-prompt](docs/bad-prompt.png)

Several strategies can take place:
* Clean Queries
* Is the query properly contextualized? Follow up questions
* MultiQueryRetrieval


In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
from typing import List, TypedDict
from dotenv import load_dotenv

from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_qdrant import QdrantVectorStore
from langchain_core.prompts import ChatPromptTemplate

from src import utils, conf

c:\Users\manuelalberto.romero\Documents\repos\dslabs\dslab-rag-e2e\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Params

In [3]:
conf_settings = conf.load(file="settings.yaml")
conf_infra = conf.load(file="infra.yaml")    

LLM_WORKHORSE = conf_settings.llm_workhorse
LLM_FLAGSHIP = conf_settings.llm_flagship
EMBEDDINGS = conf_settings.embeddings
VDB_URL = conf_infra.vdb_url
INDEX_NAME = conf_settings.vdb_index


# Environment Variables

In [4]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")

# Clients

In [5]:
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model=LLM_WORKHORSE,
    )
try:
    _ = llm.invoke("tell me a joke about devops")
except Exception as err:
    print(err)

    
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model=EMBEDDINGS)
try:
    _ = embeddings.embed_query("healthcheck")

except Exception as err:
    print(err)



vector_store = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    collection_name=INDEX_NAME,
    url=VDB_URL,
    api_key=QDRANT_API_KEY,
)
try:
    _ = vector_store.asimilarity_search("healthcheck")
except Exception as err:
    print(err)




In [6]:
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model=LLM_WORKHORSE,
    temperature=0.5
    )



# Cleaning

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from pydantic import BaseModel, Field
from enum import Enum

    


qrewrite_template = """You are an assistant tasked with taking a natural languge query from a user
    and converting it into a query for a vectorstore. In the process, strip out all 
    information that is not relevant for te retrieval task and return a new, simplified
    question for vectorstore retrieval.
    For example:
    * Fix typing mistakes
    * Remove irrelevant information for the retrieval task
    Question: {question} """

prompt_qrewrite= ChatPromptTemplate.from_template(qrewrite_template)
chain_qrewrite = prompt_qrewrite | llm

chain_qrewrite.invoke(
    {
        "question": "I love nature. What are the primary causes of the decline in bee populations globally?",
    }
)


AIMessage(content='What are the primary causes of the decline in bee populations globally?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 108, 'total_tokens': 121, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_75546bd1a7', 'id': 'chatcmpl-D8vy1A3kEanbqXDgvG6JOqNVTmZPl', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c5920-69a3-7021-b7a5-69e68eb400b8-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 108, 'output_tokens': 13, 'total_tokens': 121, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [8]:
q_raw1 = "Que telescopio descubrio Althera. Di si fue un telescopio optico o infrarrojo o de otra clase"

q_ref1 = chain_qrewrite.invoke({"question": q_raw1})
q_ref1

AIMessage(content='Qué telescopio descubrió Althera y de qué tipo era (óptico, infrarrojo u otro)', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 116, 'total_tokens': 140, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_a391f2cee0', 'id': 'chatcmpl-D8vy2s70k6w8btKCyLxwEpzoP4Q8s', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c5920-6daf-7822-a5bc-fc27d1f7d066-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 116, 'output_tokens': 24, 'total_tokens': 140, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [9]:
q_raw2 = "Quien descubrió Althera, la NASA o la ESA"

q_ref2 = chain_qrewrite.invoke({"question": q_raw2})
q_ref2

AIMessage(content='¿Quién descubrió Althera?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 104, 'total_tokens': 112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_75546bd1a7', 'id': 'chatcmpl-D8vy3jjVVDgesfa4z5Q4wNqPn6AoW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c5920-7248-78e1-a79c-cf431e937e73-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 104, 'output_tokens': 8, 'total_tokens': 112, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [10]:
from rag import main as rag_agent_builder

rag_graph = rag_agent_builder("space", 3)


resp = rag_graph.invoke({"question": q_raw1})
resp['answer']

C:\Users\manuelalberto.romero\AppData\Local\Temp\ipykernel_19732\2193883185.py:3: RuntimeWarning: coroutine 'VectorStore.asimilarity_search' was never awaited
  rag_graph = rag_agent_builder("space", 3)


'El sistema binario Althéra (HD 4579 AB) fue detectado por primera vez en el año 2032 durante una campaña de observación del Observatorio Espacial James Webb. Este telescopio es un telescopio infrarrojo.'

In [11]:
resp = rag_graph.invoke({"question": q_ref1.content})
resp['answer']

'El telescopio que descubrió el sistema binario Althéra fue el Observatorio Espacial James Webb, que es un telescopio de tipo infrarrojo.'

# Follow up

In [12]:
from typing import Optional  
from pydantic import BaseModel, Field


class Contextualizer(BaseModel):
    """ 
    Determine whether a user query is properly contextualized for retrieval.
    """
    is_contextualized: bool = Field(..., description="True if the query is properly contextualized, False otherwise")
    query_examples: Optional[str]  = Field(..., description="If not properly contextualized, provide a list of similar questions that you consider properly contextualized")
    


qfollowup_template = """You are an assistant tasked with taking a natural languge query from a user
    and evaluating whether it is properly contextualized to perform a retrieval task.
    -----
    Examples:
    Question: ¿Que seguro es más barato?
    is_contextualized: False
    query_examples: [¿Que seguro de coche es más barato?, ¿Que seguro de autos es más barato?, ¿Que seguro de hogar es mas economico?]
    ------
    Question: {question} """

prompt_qfollowup= ChatPromptTemplate.from_template(qfollowup_template)
chain_qfollowup = prompt_qfollowup | llm.with_structured_output(Contextualizer )

chain_qfollowup.invoke(
    {
        "question": "function string to datetime",
    }
)


Contextualizer(is_contextualized=False, query_examples='[¿Cómo convertir una cadena a un objeto datetime en Python?, ¿Función para transformar string a datetime en JavaScript?, ¿Cómo parsear una fecha desde un string en Java?]')

# MultiQuery Retrieval


![multiqueryrewrite.png](docs/multiqueryrewrite.png)

In [16]:
from langchain_classic.retrievers.multi_query import MultiQueryRetriever

from langchain_openai import ChatOpenAI

question = "¿Qué observatorio confirmó la existencia de cinco planetas principales?"
llm = ChatOpenAI(temperature=0.4)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(),
    llm=llm,
    include_original=True
)

import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

retriever_from_llm.invoke(question)

[Document(metadata={'Header 1': '1. Historia del descubrimiento', '_id': '58b2f577-358a-4e6c-8ffd-8696135b7c3a', '_collection_name': 'space'}, page_content='# 1. Historia del descubrimiento  \n## 1.1 Primeras observaciones y sospechas iniciales  \nEl sistema binario Althéra ( HD 4579 AB ) fue detectado por primera vez en el año 2032 durante una campaña de observación del Observatorio Espacial James Webb , dirigida por la astrofísica chilena Dra. Mariela Estay . La misión principal era estudiar la composición atmosférica de exoplanetas candidatos a la habitabilidad, pero un patrón anómalo en el flujo luminoso proveniente de la constelación de Orión llamó la atención del equipo. El análisis de curvas de luz reveló oscilaciones periódicas dobles, un indicio claro de la presencia de dos estrellas en órbita mutua y varios cuerpos orbitando de forma circumbinaria.  \n## 1.2 Confirmación mediante técnicas combinadas  \nEn los meses siguientes, un consorcio internacional liderado por la Agenci

In [17]:
question = "¿Qué instrumento confirmó la existencia de cinco planetas principales?"

retriever_from_llm.invoke(question)

[Document(metadata={'Header 1': '1. Historia del descubrimiento', '_id': '58b2f577-358a-4e6c-8ffd-8696135b7c3a', '_collection_name': 'space'}, page_content='# 1. Historia del descubrimiento  \n## 1.1 Primeras observaciones y sospechas iniciales  \nEl sistema binario Althéra ( HD 4579 AB ) fue detectado por primera vez en el año 2032 durante una campaña de observación del Observatorio Espacial James Webb , dirigida por la astrofísica chilena Dra. Mariela Estay . La misión principal era estudiar la composición atmosférica de exoplanetas candidatos a la habitabilidad, pero un patrón anómalo en el flujo luminoso proveniente de la constelación de Orión llamó la atención del equipo. El análisis de curvas de luz reveló oscilaciones periódicas dobles, un indicio claro de la presencia de dos estrellas en órbita mutua y varios cuerpos orbitando de forma circumbinaria.  \n## 1.2 Confirmación mediante técnicas combinadas  \nEn los meses siguientes, un consorcio internacional liderado por la Agenci

In [18]:
question = "DAG en Airflow"

retriever_from_llm.invoke(question)

[Document(metadata={'Header 1': '2. Conoce a Althéra', '_id': '31808336-1343-4b2c-b3b7-86fe7e20fec3', '_collection_name': 'space'}, page_content='Entre sus rasgos distintivos:  \n- Luminiscencia variable combinada: las diferencias de color y brillo entre Althéra A y B generan amaneceres y atardeceres dobles de tonalidades doradas y anaranjadas.\n- Influencia gravitacional múltiple: la presencia de dos gigantes gaseosos exteriores, Zephyros IV y Krion V, ayuda a limpiar el espacio interior de objetos errantes, protegiendo a los planetas habitables de impactos masivos frecuentes.\n- Cinturón de Arges activo: región rica en asteroides metálicos, con colisiones frecuentes que producen brillantes lluvias de meteoros observables desde Aurelia III.  \nEn conjunto, Althéra constituye un laboratorio natural para estudiar cómo la vida -si existe o llegara a desarrollarse- podría adaptarse a condiciones lumínicas, gravitacionales y climáticas mucho más complejas que las de nuestro sistema solar.'

```python 

# Default prompt
DEFAULT_QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is
    to generate 3 different versions of the given user
    question to retrieve relevant documents from a vector  database.
    By generating multiple perspectives on the user question,
    your goal is to help the user overcome some of the limitations
    of distance-based similarity search. Provide these alternative
    questions separated by newlines. Original question: {question}""",
)


```


https://python.langchain.com/api_reference/_modules/langchain/retrievers/multi_query.html#MultiQueryRetriever

In [19]:
qexpa_template = """You are an assistant tasked with taking a natural languge query from a user
    and generating more suitable candidates to perform a retrieval task.
    Generate 3 different versions withouth changing the main goal of the question
    Focus on:
    * looking synonyms, related terms or adding explanations of key concepts in the query
    * Resolve well known acronyms
    * Translate terms to the original query language
    Provide these alternative questions separated by newlines. 
    -----
    Examples:
    Question: Is Python a good programming language?
    query_examples: [Is Python a performant programming language?, Does Python manages memory efficiently?, Is Python a good programming language for data science?]
    ------
    Original question: {question} """

# a good example
# https://smith.langchain.com/hub/smithing-gold/question-decomposition

prompt_qexpa = ChatPromptTemplate.from_template(qexpa_template)


question = "¿Qué observatorio confirmó la existencia de cinco planetas principales?"
retriever_from_llm_custom = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(),
    llm=llm,
    include_original=True,
    prompt=prompt_qexpa

)

retriever_from_llm_custom.invoke(question)

[Document(metadata={'Header 1': '3. Los soles de Althéra', '_id': '3fcbb7a9-2a0d-4784-b473-10ad54f03f4c', '_collection_name': 'space'}, page_content='## 3.3 Ciclo de vida y edad estimada del sistema binario  \nEstudios de metalicidad realizados con el ELT indican que ambas estrellas tienen composiciones químicas similares, con una proporción de elementos pesados algo mayor que la del Sol ([Fe/H] ≈ +0,08). Esto sugiere que se formaron a partir de la misma nube molecular hace aproximadamente 4.900 millones de años , una edad comparable a la del Sistema Solar.  \nAlthéra A, al ser más masiva, evolucionará hacia gigante roja en unos 4.000 millones de años, mientras que Althéra B permanecerá estable durante varios miles de millones más. Esta diferencia temporal tendrá un fuerte impacto en la evolución futura de los planetas circumbinarios.  \n## 3.4 Interacción gravitatoria y efectos sobre la zona habitable  \nLa órbita mutua de las dos estrellas, con una separación media de 0,42 UA y una e

In [20]:
question = "¿Qué instrumento confirmó la existencia de cinco planetas principales?"
retriever_from_llm_custom.invoke(question)

[Document(metadata={'Header 1': '1. Historia del descubrimiento', '_id': '58b2f577-358a-4e6c-8ffd-8696135b7c3a', '_collection_name': 'space'}, page_content='# 1. Historia del descubrimiento  \n## 1.1 Primeras observaciones y sospechas iniciales  \nEl sistema binario Althéra ( HD 4579 AB ) fue detectado por primera vez en el año 2032 durante una campaña de observación del Observatorio Espacial James Webb , dirigida por la astrofísica chilena Dra. Mariela Estay . La misión principal era estudiar la composición atmosférica de exoplanetas candidatos a la habitabilidad, pero un patrón anómalo en el flujo luminoso proveniente de la constelación de Orión llamó la atención del equipo. El análisis de curvas de luz reveló oscilaciones periódicas dobles, un indicio claro de la presencia de dos estrellas en órbita mutua y varios cuerpos orbitando de forma circumbinaria.  \n## 1.2 Confirmación mediante técnicas combinadas  \nEn los meses siguientes, un consorcio internacional liderado por la Agenci

In [21]:
question = "¿Fue la NASA quien descubrió Althera?"
retriever_from_llm_custom.invoke(question)

[Document(metadata={'_id': 'b2c0cbc8-2249-4740-b46e-2b70a0335342', '_collection_name': 'space'}, page_content='## Un nuevo y fascinante vecino: Althéra  \n## Índice  \n1. Historia del descubrimiento\n2. Conoce a Althéra\n3. Los soles de Althéra\n4. Estructura general de Althéra\n5. Planetas interiores\n6. Planetas exteriores\n7. Lunas y satélites menores\n8. Fenómenos destacados\n9. Habitabilidad y astrobiología\n10. Conclusiones y perspectivas futuras'),
 Document(metadata={'Header 1': '1. Historia del descubrimiento', '_id': 'afa12015-551f-4838-8c51-992a215428dd', '_collection_name': 'space'}, page_content='## 1.3 Descubrimiento revolucionario de la zona habitable circumbinaria  \nEl hallazgo más impactante llegó en 2034, cuando la misión LUVOIR-B (Large UV/Optical/IR Surveyor) detectó la firma espectral de vapor de agua, oxígeno molecular y metano en la atmósfera de Aurelia III , un planeta ubicado en la zona habitable del sistema, orbitando a ambos soles. Este fue el primer caso do

In [22]:
question = "¿Fue la ESA quein descubrió Althera?"
retriever_from_llm_custom.invoke(question)

[Document(metadata={'Header 1': '1. Historia del descubrimiento', '_id': 'afa12015-551f-4838-8c51-992a215428dd', '_collection_name': 'space'}, page_content='## 1.3 Descubrimiento revolucionario de la zona habitable circumbinaria  \nEl hallazgo más impactante llegó en 2034, cuando la misión LUVOIR-B (Large UV/Optical/IR Surveyor) detectó la firma espectral de vapor de agua, oxígeno molecular y metano en la atmósfera de Aurelia III , un planeta ubicado en la zona habitable del sistema, orbitando a ambos soles. Este fue el primer caso documentado de un mundo potencialmente habitable en un sistema binario cercano -a tan solo 42,7 años luz de la Tierra -, lo que lo convierte en un candidato ideal para futuras misiones de exploración interestelar.  \n## 1.4 Importancia científica y proyección futura  \nEl descubrimiento de Althéra revolucionó la astrobiología y la física orbital por tres razones clave:  \n1. Dinámica circumbinaria estable - demostró que los planetas pueden mantener órbitas e